In [1]:
# Avocado prices.
from bs4 import BeautifulSoup
import pandas as pd
import os
import re 
import json
import requests
from pandas.io.json import json_normalize


avo = pd.read_csv('avocado_sin_key.csv')
avo = pd.DataFrame(avo)
avo['date']= pd.to_datetime(avo['date'], format = '%d/%m/%Y').dt.date
'''
 INTRODUCCIÓN A LA BASE.
 La base de datos es de la venta de aguacate (orgánico y convencional) en supermercados de estados unidos,
 las observaciones se refieren a semanas del 27-12-2015 al ______________________
 Las semanas se repiten ya que de cada region, se registraron las mismas semanas.
 De las catorce variables, una es de tipo entero (year), tres de tipo categórico (Date, type, region) 
 y las demás son flotante.
'''

'\n INTRODUCCIÓN A LA BASE.\n La base de datos es de la venta de aguacate (orgánico y convencional) en supermercados de estados unidos,\n las observaciones se refieren a semanas del 27-12-2015 al ______________________\n Las semanas se repiten ya que de cada region, se registraron las mismas semanas.\n De las catorce variables, una es de tipo entero (year), tres de tipo categórico (Date, type, region) \n y las demás son flotante.\n'

In [2]:
# Pequeño análisis de la constitución de la base
len(avo) # 17911 observaciones
len(avo.columns) # 14 columnas
avo.isnull().sum() # sin datos faltantes


Unnamed         0
date            0
Price_us        0
total_volume    0
4046            0
4225            0
4770            0
total_bags      0
small_bags      0
large_bags      0
xl_bags         0
type            0
year            0
region          0
dtype: int64

In [3]:

'''
Objetivo.
Lo que se quiere realizar es mostrar el precio promedio en pesos mexicanos y además ver la latitud y la longitud de
las regiones de venta.
Para el precio promedio en pesos, se va a extraer la información de la API de Banxico.
'''

'\nObjetivo.\nLo que se quiere realizar es mostrar el precio promedio en pesos mexicanos y además ver la latitud y la longitud de\nlas regiones de venta.\nPara el precio promedio en pesos, se va a extraer la información de la API de Banxico.\n'

In [6]:

with open('.gitignore') as f:
    acceso = f.read()
prueba = acceso.strip()


In [10]:

response = requests.get('https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF63528/datos/2015-01-04/2018-03-26', headers = {'Bmx-Token':prueba})
result = response.json()
# Como result arrojó diccionarios de diccionarios de una sóla key, entonces buscamos llegar a la lista de diccionarios
# donde estuvieran las fechas y la cantidad, para poder normalizar el json.

tc_date = result['bmx']['series'][0]['datos']
df_tc = json_normalize(tc_date)
df_tc['fecha']= pd.to_datetime(df_tc['fecha'], format = '%d/%m/%Y').dt.date
df_tc

,dato,fecha
0,14.94690,2015-01-05
1,14.84790,2015-01-06
2,14.79360,2015-01-07
3,14.62740,2015-01-08
4,14.58040,2015-01-09
5,14.63980,2015-01-12
6,14.55590,2015-01-13
7,14.56440,2015-01-14
8,14.58210,2015-01-15
9,14.62230,2015-01-16


In [9]:
df_tc.to_csv('df_tc.csv')

In [37]:
tc = []
i, j = 0,0

while i < len(avo) and j < len(df_tc):
    print(avo['date'][i],i,df_tc['fecha'][j],j)
    if avo['date'][i] == df_tc['fecha'][j]:
        tc.append(float(df_tc['dato'][j]))
        i += 1
    elif avo['date'][i] < df_tc['fecha'][j]:
        tc.append('No esta {}'.format(avo['date'][i]))
        i += 1
    else:
        j += 1


2015-01-05 0 2015-01-05 0
2015-01-05 1 2015-01-05 0
2015-01-05 2 2015-01-05 0
2015-01-05 3 2015-01-05 0
2015-01-05 4 2015-01-05 0
2015-01-05 5 2015-01-05 0
2015-01-05 6 2015-01-05 0
2015-01-05 7 2015-01-05 0
2015-01-05 8 2015-01-05 0
2015-01-05 9 2015-01-05 0
2015-01-05 10 2015-01-05 0
2015-01-05 11 2015-01-05 0
2015-01-05 12 2015-01-05 0
2015-01-05 13 2015-01-05 0
2015-01-05 14 2015-01-05 0
2015-01-05 15 2015-01-05 0
2015-01-05 16 2015-01-05 0
2015-01-05 17 2015-01-05 0
2015-01-05 18 2015-01-05 0
2015-01-05 19 2015-01-05 0
2015-01-05 20 2015-01-05 0
2015-01-05 21 2015-01-05 0
2015-01-05 22 2015-01-05 0
2015-01-05 23 2015-01-05 0
2015-01-05 24 2015-01-05 0
2015-01-05 25 2015-01-05 0
2015-01-05 26 2015-01-05 0
2015-01-05 27 2015-01-05 0
2015-01-05 28 2015-01-05 0
2015-01-05 29 2015-01-05 0
2015-01-05 30 2015-01-05 0
2015-01-05 31 2015-01-05 0
2015-01-05 32 2015-01-05 0
2015-01-05 33 2015-01-05 0
2015-01-05 34 2015-01-05 0
2015-01-05 35 2015-01-05 0
2015-01-05 36 2015-01-05 0
2015-01-05 

In [38]:
avo['exchange_rate'] = tc

In [39]:
set(avo['exchange_rate'])

{14.588699999999999,
 14.639799999999999,
 14.6524,
 14.756600000000001,
 14.8003,
 14.801399999999999,
 14.8979,
 14.946899999999999,
 14.964399999999999,
 14.9862,
 15.0832,
 15.119999999999999,
 15.242699999999999,
 15.2624,
 15.263500000000001,
 15.294700000000001,
 15.3072,
 15.321300000000001,
 15.3973,
 15.427099999999999,
 15.432700000000001,
 15.4544,
 15.484500000000001,
 15.494,
 15.6599,
 15.6715,
 15.740399999999999,
 15.753,
 15.9856,
 16.1129,
 16.165199999999999,
 16.2681,
 16.3993,
 16.419599999999999,
 16.421900000000001,
 16.489899999999999,
 16.4998,
 16.5306,
 16.585599999999999,
 16.686499999999999,
 16.694099999999999,
 16.757000000000001,
 16.782900000000001,
 16.816800000000001,
 16.870000000000001,
 16.904399999999999,
 16.988,
 17.007300000000001,
 17.0975,
 17.103400000000001,
 17.227900000000002,
 17.271000000000001,
 17.336099999999998,
 17.352900000000002,
 17.377600000000001,
 17.448399999999999,
 17.459800000000001,
 17.460999999999999,
 17.583600000000

In [42]:
avo['Price_mx']= avo['Price_us']*avo['exchange_rate']
avo['Price_mx']


0        26.157075
1        22.270881
2        25.110792
3        22.719288
4        24.512916
5        11.210175
6        12.704865
7        22.420350
8        14.946900
9        11.957520
10       11.060706
11       27.203358
12       15.096369
13       26.904420
14       27.053889
15       17.188935
16       25.708668
17       23.317164
18       15.694245
19       15.245838
20       17.786811
21       14.946900
22       26.754951
23       18.235218
24       15.096369
25       14.797431
26       14.199555
27       17.936280
28       25.858137
29       15.694245
           ...    
17881    24.008108
17882    17.593728
17883    30.422488
17884    33.354776
17885    23.824840
17886    23.458304
17887    32.438436
17888    10.812812
17889    25.657520
17890    34.087848
17891    29.139612
17892    37.020136
17893    23.458304
17894    22.541964
17895    19.792944
17896    18.510068
17897    24.008108
17898    18.876604
17899    24.924448
17900    26.573860
17901    25.474252
17902    32.

In [10]:
# REGIONES

In [11]:
# Lo que vamos a hacer es separar los nombres de las regiones por las mayúsculas, además vamos a cambiar el nombre
# de Total U s a US
reg_sep = list(map((lambda i: re.findall('[A-Z][^A-Z]*', i)),avo['region']))
prue=[]
for i in reg_sep:
    if len(i)>1:
        prue.append(' '.join(i))
    else:
        prue.append(i[0])
avo['region']= prue

In [12]:
for i in range(len(avo['region'])):
    if avo['region'][i] == 'Total U S':
        avo['region'][i] = 'US'

/Users/valeriajimeno/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [13]:
# En la variable de regiones, vienen además de regiones, ciudades, por lo que para buscar la latitud y longitud,
# vamos a quitar las regiones y dejaremos sólo las ciudades.

avo = avo.drop(avo[(avo.region == 'Northeast')|(avo.region == 'Midsouth')|(avo.region == 'South Central')|(avo.region == 'Southeast')|(avo.region == 'West')|(avo.region == 'US')|(avo.region == 'Plains')|(avo.region == 'California')|(avo.region == 'Great Lakes')|(avo.region == 'South Carolina')|(avo.region == 'Northern New England')|(avo.region =='West Tex New Mexico')].index)


In [14]:
# WebS Scrapping

# Ahora para saber la latitud y longitud, vamos a scrapear de la siguiente página la tabla con las latitudes de
# las 1000 ciudades más importantes de US, para poder hacer merge con la columna de regiones
# y así hacer una columna con la latitud y otra con longitud.

request = requests.get('https://www.latlong.net/category/cities-236-15.html').content
soup = BeautifulSoup(request, 'lxml')
tabla = soup.find_all('table')
tab_sucia = [tabla[i].text.split('\n\n') for i in range(len(tabla))][0]

In [15]:
tab = [i.replace('USA','US').replace('UK','US').split(',') for i in tab_sucia]
tab_clean = [(i[0].strip(),i[-1].replace(' the US','').replace('US','').strip()) for i in tab]
lat_long = pd.DataFrame(tab_clean, columns=['country', 'lat-long',]).drop_duplicates(subset = ['country'], keep='first')

In [16]:
lat_long = lat_long.sort_values(by =['country']).reset_index(drop=True)

In [17]:
# limpiar la columna de regiones
def ciudades(i):
    if i == 'Baltimore Washington':
        i = 'Baltimore'
    elif i == 'Buffalo Rochester':
        i = 'Buffalo'
    elif i == 'Cincinnati Dayton':
        i = 'Cincinnati'
    elif i == 'Dallas Ft Worth':
        i = 'Dallas'
    elif i == 'Harrisburg Scranton':
        i = 'Harrisburg'
    elif i == 'Hartford Springfield':
        i = 'Springfield'
    elif i == 'Miami Ft Lauderdale':
        i = 'Miami'
    elif i == 'New Orleans Mobile':
        i = 'New Orleans'
    elif i == 'Phoenix Tucson':
        i = 'Phoenix'
    elif i == 'Raleigh Greensboro':
        i = 'Raleigh'
    elif i == 'Richmond Norfolk':
        i = 'Richmond'
    elif i == 'St Louis':
        i = 'St. Louis'
    elif i == 'New York':
        i = 'New York City'
        i = i 
    return i


In [18]:
avo.region =avo.region.apply(ciudades)

In [19]:
avo.head()

,Unnamed,date,Price_us,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xl_bags,type,year,region,tipo_cambio
1,51,2015-01-05,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.00,organic,2015,Chicago,14.94690
2,51,2015-01-05,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.00,organic,2015,Harrisburg,14.94690
3,51,2015-01-05,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.00,conventional,2015,Pittsburgh,14.94690
4,51,2015-01-05,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.00,organic,2015,Boise,14.94690
6,51,2015-01-05,0.85,2682159.95,1837999.65,524430.47,64225.78,255504.05,215571.80,36981.72,2950.53,conventional,2015,Los Angeles,14.94690


In [20]:
avo = avo.sort_values(by = ['region']).reset_index(drop=True)
len(avo)

13858

In [21]:
i,j =0,0
coordinates = []

while i < len(avo) and j < len(lat_long):
    if avo.region[i] == lat_long.country[j]:
        coordinates.append(lat_long['lat-long'][j])
        i += 1
    elif avo.region[i] > lat_long.country[j]:
        j += 1
    else:
        coordinates.append('No está {}'.format(avo.region[i]))
        i += 1

In [22]:
coordinates

['44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',
 '44.636513-123.105927',


In [23]:
avo['coordinates']=coordinates

In [27]:
set(avo.tipo_cambio)

{'14.58870',
 '14.63980',
 '14.65240',
 '14.75660',
 '14.80030',
 '14.80140',
 '14.89790',
 '14.94690',
 '14.96440',
 '14.98620',
 '15.08320',
 '15.12000',
 '15.24270',
 '15.26240',
 '15.26350',
 '15.29470',
 '15.30720',
 '15.32130',
 '15.39730',
 '15.42710',
 '15.43270',
 '15.45440',
 '15.48450',
 '15.49400',
 '15.65990',
 '15.67150',
 '15.74040',
 '15.75300',
 '15.98560',
 '16.11290',
 '16.16520',
 '16.26810',
 '16.39930',
 '16.41960',
 '16.42190',
 '16.48990',
 '16.49980',
 '16.53060',
 '16.58560',
 '16.68650',
 '16.69410',
 '16.75700',
 '16.78290',
 '16.81680',
 '16.87000',
 '16.90440',
 '16.98800',
 '17.00730',
 '17.09750',
 '17.10340',
 '17.22790',
 '17.27100',
 '17.33610',
 '17.35290',
 '17.37760',
 '17.44840',
 '17.45980',
 '17.46100',
 '17.58360',
 '17.58660',
 '17.64430',
 '17.64520',
 '17.65010',
 '17.68930',
 '17.75260',
 '17.76100',
 '17.76910',
 '17.77550',
 '17.80490',
 '17.86460',
 '17.86830',
 '17.87750',
 '17.88310',
 '17.93040',
 '17.95190',
 '17.96410',
 '17.97510',